## What is Pinecone?

pinecone is a managed vector database that lets you efficiently store and search through high dimensional vector embeddings . its particularly useful for applications like semantic search , recommendation systems,and conversational AI.

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
embedding_model

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000021025636150>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [4]:
vectors = embedding_model.embed_query("DataScilearn")
len(vectors)


768

In [5]:
vectors

[0.04671652987599373,
 -0.0470898263156414,
 -0.021976450458168983,
 0.008908337913453579,
 0.04835832864046097,
 0.016732819378376007,
 -0.01311538740992546,
 -0.06284113228321075,
 0.01355375163257122,
 0.015467061661183834,
 -0.008265563286840916,
 -0.026263240724802017,
 -0.04270730912685394,
 -0.06579045206308365,
 -0.0026062538381665945,
 -0.04167531430721283,
 0.017155786976218224,
 -0.01685633696615696,
 -0.0026222970336675644,
 0.0030948310159146786,
 -0.018952464684844017,
 0.0018846092280000448,
 -0.009754272177815437,
 -0.04164974391460419,
 0.006189592182636261,
 -0.04453013837337494,
 0.054738178849220276,
 -0.06845910102128983,
 -0.03105803392827511,
 0.033480141311883926,
 -0.05154556781053543,
 0.01015511155128479,
 -0.0007045857491903007,
 0.03334097936749458,
 0.023579267784953117,
 -0.03291315585374832,
 0.002977105090394616,
 0.014062500558793545,
 0.026245497167110443,
 0.003981922287493944,
 -0.03408205509185791,
 -0.040309663861989975,
 -0.04807337746024132,
 -0

In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
loader = PyPDFLoader("RAG.pdf")
data = loader.load()
data

[Document(metadata={'source': 'RAG.pdf', 'page': 0}, page_content='Adaptive-RAG: Learning to Adapt Retrieval-Augmented\nLarge Language Models through Question Complexity\nSoyeong Jeong1Jinheon Baek2Sukmin Cho1Sung Ju Hwang1,2Jong C. Park1*\nSchool of Computing1Graduate School of AI2\nKorea Advanced Institute of Science and Technology1,2\n{starsuzi,jinheon.baek,nelllpic,sjhwang82,jongpark}@kaist.ac.kr\nAbstract\nRetrieval-Augmented Large Language Models\n(LLMs), which incorporate the non-parametric\nknowledge from external knowledge bases into\nLLMs, have emerged as a promising approach\nto enhancing response accuracy in several tasks,\nsuch as Question-Answering (QA). However,\neven though there are various approaches deal-\ning with queries of different complexities, they\neither handle simple queries with unnecessary\ncomputational overhead or fail to adequately\naddress complex multi-step queries; yet, not\nall user requests fall into only one of the sim-\nple or complex categories.

In [8]:
text_spiltter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=30)
text_chunks = text_spiltter.split_documents(data)
text_chunks

[Document(metadata={'source': 'RAG.pdf', 'page': 0}, page_content='Adaptive-RAG: Learning to Adapt Retrieval-Augmented\nLarge Language Models through Question Complexity\nSoyeong Jeong1Jinheon Baek2Sukmin Cho1Sung Ju Hwang1,2Jong C. Park1*\nSchool of Computing1Graduate School of AI2\nKorea Advanced Institute of Science and Technology1,2\n{starsuzi,jinheon.baek,nelllpic,sjhwang82,jongpark}@kaist.ac.kr\nAbstract\nRetrieval-Augmented Large Language Models\n(LLMs), which incorporate the non-parametric\nknowledge from external knowledge bases into'),
 Document(metadata={'source': 'RAG.pdf', 'page': 0}, page_content='LLMs, have emerged as a promising approach\nto enhancing response accuracy in several tasks,\nsuch as Question-Answering (QA). However,\neven though there are various approaches deal-\ning with queries of different complexities, they\neither handle simple queries with unnecessary\ncomputational overhead or fail to adequately\naddress complex multi-step queries; yet, not\nall use

In [10]:
text_chunks[1].page_content

'LLMs, have emerged as a promising approach\nto enhancing response accuracy in several tasks,\nsuch as Question-Answering (QA). However,\neven though there are various approaches deal-\ning with queries of different complexities, they\neither handle simple queries with unnecessary\ncomputational overhead or fail to adequately\naddress complex multi-step queries; yet, not\nall user requests fall into only one of the sim-\nple or complex categories. In this work, we'

In [11]:
os.environ['PINECONE_API_KEY'] = os.getenv("PINECONE_API_KEY")

In [12]:
index_name = "test"

In [13]:
from langchain_pinecone import PineconeVectorStore
vectorstore_from_docs = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=index_name,
    embedding=embedding_model
)

In [14]:
query = "What is single step approach"
vectorstore_from_docs.similarity_search(query,k=2)

[Document(id='016db2c9-3cf6-44f4-bbd7-c81d16746789', metadata={'page': 1.0, 'source': 'RAG.pdf'}, page_content='a query, this single-step approach retrieves relevant documents and then generates an answer. However, it may not be sufficient\nfor complex queries that require multi-step reasoning. (B) This multi-step approach iteratively retrieves documents and generates\nintermediate answers, which is powerful yet largely inefficient for the simple query since it requires multiple accesses to both'),
 Document(id='4e98ef10-8a2d-494d-9c83-8e6197840f08', metadata={'page': 6.0, 'source': 'RAG.pdf'}, page_content='sample and annotate 400 queries from 6 datasets\nbased on its inductive bias (single-hop for one-step\napproach and multi-hop for multi-step). In addition,\nwe use predicted outcomes of three different strate-\ngies over 400 queries sampled from each dataset.\nNote that those queries used for classifier training\ndo not overlap with the testing queries for QA.\n5 Experimental Resul

In [15]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

qa_chain = RetrievalQA.from_chain_type(llm=llm,chain_type='stuff',retriever=vectorstore_from_docs.as_retriever())

In [16]:
query = "what is single-step approach"

In [17]:
qa_chain.run(query)

C:\Users\pkkar\AppData\Local\Temp\ipykernel_28340\1351296832.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run(query)


'The single-step approach is a method for answering queries that involves retrieving relevant documents and then generating an answer in a single step. This approach is suitable for simple queries that can be answered directly from a single document. \n'

In [20]:
qa_chain.invoke(query)

{'query': 'what is single-step approach',
 'result': 'The single-step approach is a method for answering queries that involves retrieving relevant documents and then generating an answer in a single step. This approach is suitable for simple queries that do not require multi-step reasoning. \n'}